# More On Word Embeddings


As we found at the end of the Notebook 2, we only use only 80% of information in average and discard the rest. In this experiment, we will explore how can we use information from the dataset more efficiently i.e. reduce the fraction of discard information. Although we still focus on word embeddings, we will tackle it differently this time. 

First let's recall from the Notebook 2 the reason why the efficiency is not very high. In that experiment, we vectorize  text by getting word embeddings corresponding to each tokens from pre-trained word embeddings and then polling to one single vector representing the text. For tokens that we don't have corresponding word embeddings, we randomly pick one from the pre-trained word embeddings (or we can simply ignore). Because there are some tokens, approximately 20%, that do not present in our pre-trained word embeddings, it means that we discard those tokens and so the model will never learn from this information.

One way to solve this problem is to train word embeddings using the train set (we can't use both train and test to train word embeddings due to data leakage). This way we will every token in train set a corresponding vector. We may still observe unknown tokens in test set, but it will be much less than what we observed in Notebook 2.

This [paper](https://www.aclweb.org/anthology/Q15-1016.pdf) discusses various aspects of word embeddings on different tasks, as well as hyper parameters tuning. However, they didn't benchmark on text classification task as our experiment.

Now let's discuss the experiments.


We have quite several variations on implementation. Let's discuss them one by one.




**Prerequisite**

1. Download [Google Word2Vec Model](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing) to this directory and run 

    ```
    gunzip GoogleNews-vectors-negative300.bin.gz
    ```

2. Download [Stanford GloVe Model](http://nlp.stanford.edu/data/glove.840B.300d.zip) to this directory and run the following commands.

    ```
    unzip glove.840B.300d.zip
    python -m gensim.scripts.glove2word2vec --input glove.840B.300d.txt --output glove.840B.300d.w2vformat.txt
    ```

GloVe is also available in SpaCy's `en_core_web_md` too. See [Document](https://spacy.io/models/en#en_core_web_md). In this notebook, we will not use GloVe from SpaCy due to lots of its limitations.

If you already have those files or you don't want to save it in this directory, you can either change constant variable PRETRAINED_WV_MODEL_PATH  and PRETRAINED_GLOVE_MODEL_PATH or create symbolic link.
    
```
ln -s /path/to/your/word2vec ./GoogleNews-vectors-negative300.bin
ln -s /path/to/your/glove ./glove.840B.300d.w2vformat.txt

```

In [1]:
%load_ext autoreload
%autoreload

from lib.dataset import download_tfds_imdb_as_text, download_tfds_imdb_as_text_tiny
from lib.word_emb import run_pipeline, train_or_load_wv, train_or_load_wv_transfer




In [2]:
dataset  = download_tfds_imdb_as_text() # tuple of (X_train, X_test, y_train, y_test)
tiny_dataset = download_tfds_imdb_as_text_tiny() # first 100 samples from dataset

PRETRAINED_WV_MODEL_PATH = "./GoogleNews-vectors-negative300.bin"

## Baseline

Before starting the experiment, let's train the Word2Vec model with all default settings, pass it to the same text classification `pipeline` discussed in Notebook  2 and save the result as baseline.

Hyperparameters:
- Dimension `dim = 300`
- Window Size `window = 5`
- Epochs `iter = 5`


In [3]:
model_train = train_or_load_wv(dataset[0]) 
_, vectorizer = run_pipeline(dataset, model_train)

Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.87
F1 on test set: 0.87
time: 576.78


In [10]:
vectorizer.print_stat()

fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.0048322335573936295
%unk in vocab 0.036029426544190064


We get F1 score of 0.86, slightly better than what we get from experiment 2. And we see that now we efficiently use information from dataset, as we observe 0% unknown words during training (of course because embeddings are trained on the same dataset) and only 0.4% during testing.

## Window Size

Generally speaking, window size is a size of context of consideration. For example, sentence `... We can address this issue by introducing ...  `, let's focus at word `this`. If the context size = 2, the model will try to encode the meaning of word `this` by considering some words in `can`, `address`, `issue`, `by`. It will not take `we` into account because it is out of considered context. For more details, see the [original paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). The intuition behind is this is that training the model with smaller size of windows will give you embeddings that encode more syntactic meaning. It will learn that words like `is`, `was` are similar because it's replaceable within a small window. The larger window size tends to create embeddings that can encode broader idea or topic or words. See [this paper](https://levyomer.files.wordpress.com/2014/04/dependency-based-word-embeddings-acl-2014.pdf) for more details. Whichever better is better depends on the downstream task. For example, one can say that smaller window size can perform better for downstream task like analogy, while larger size can be better for text classification. We will do experiment and see if it is the case.

Now let increase the window size from 5 to 15 and 30.

In [4]:
# approximate running time: 12 mins

model_train_window_15 = train_or_load_wv(dataset[0],window=15)
_, _ = run_pipeline(dataset, model_train_window_15)

model_train_window_30 = train_or_load_wv(dataset[0], window=30)
_, _ = run_pipeline(dataset, model_train_window_30)


Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
time: 344.16
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
time: 318.35


We got F1 score of 0.88 for window = 15, compare to 0.87 in baseline (window=5) which are slightly better. Further increasing from 15 to 30 doesn't help. We can say that at window=15, embeddings can capture "broad enough" concept for text classification.

## Epochs

Epochs is a little tricker than window size. In general, the more epochs can lead to overfitting. However, does overfitting present in Word2Vec, or does concept of overfitting make sense at all for Word2Vec? (Don't mess up with text classification model, which absolutely can be overfit). Let's go back to the fundamental concept of overiftting - overfitting occurs when a function (model) is too closely fit to a train set and are unable to generalize to other data points e.g. data points in test set. In Word2Vec (as well as other language models), we use text corpus for both training samples and thier labels. For Word2Vec, the training samples are the center words and labels are context word, (for SG, opposite for C-BOW. For traditional language model, training data can be words 0th to nth and labels can be n+1th word. Cutting-edge language models like BERT and GPT-2 use slightly different apporaches, but the fundamental idea is the same. So, can language model overfit? Can language model predict a word given some other words very well but perform poorly on data points that it hasn't been trained on? Yes, of course. Let's narrow down the question: what happen to embeddings if the Word2Vec model is overfit? Will it undermine the classification model? We will experiment on that. See [this paper](https://arxiv.org/pdf/1803.05651.pdf#:~:text=Standard%20Word2Vec%20may%20be%20prone,a%20regu%2D%20larizer%20against%20it.&text=Word%20vectors%20are%20continuous%20representations,most%20deep%20learning%20NLP%20models.) for more info.

We vary epoch from 2, 10 and 30. Recall the baseline has epoch = 5.

In [5]:
# approximate running time: 40 mins

model_train_iter_2 = train_or_load_wv(dataset[0], iter=2, window=15)
_, _ = run_pipeline(dataset, model_train_iter_2)

model_train_iter_10 = train_or_load_wv(dataset[0],iter=10, window=15)
_, _ = run_pipeline(dataset, model_train_iter_10)

model_train_iter_15 = train_or_load_wv(dataset[0], iter=30, window=15)
_, _ = run_pipeline(dataset, model_train_iter_15)



Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.88
F1 on test set: 0.87
time: 737.02
Best parameters set found on development set:  {'C': 10}
Best F1 on development set: 0.89
F1 on test set: 0.88
time: 305.73
Best parameters set found on development set:  {'C': 10}
Best F1 on development set: 0.89
F1 on test set: 0.87
time: 300.70


Training with smaller number of epochs (5 or less) tends to give the embeddings that do not encode enough information for text classification, as we observe lower F1 (0.87). Training with larger number of epochs (30) can result in embeddings that encode information that is very specific to the train dataset and not generalize to test dataset, as we observe 0.02 gap between train and test F1. We "indirectly" worsen the classification model. The optimal value for this example is 10 epochs.

## Dimension 

Dimension of embeddings can be roughly think of the capacity of information. It's, however, not necessary, let's say we have 10k but after doing PCA we find that only first one or two principal component carry information. Also, sparse representation (Notebook 1) of dimension 10k does not necessary convey more information than dense representation with 300 dimension. But, generally speaking, we can say that higher dimension means higher capacity of vessel. For how much information it is in the vessel depends on how we fill it in the vessel (training process for Word2Vec, or counting process for one hot encoding, etc). If the dataset is smaller, we may need smaller dimension because we don't have that much information to encode in. Most literatures suggest dimension of size 300 is good in most case. We will experiment what is the good number for our IMDB dataset so you can apply the idea to other dataset.


We vary epoch from 100, 500 and 1000 (baseline uses 300).

In [6]:
# approximate running time: 40 mins

model_train_dim_100 = train_or_load_wv(dataset[0], size=100, window=15)
_, _ = run_pipeline(dataset, model_train_dim_100)

model_train_dim_500 = train_or_load_wv(dataset[0], size=500, window=15)
_, _ = run_pipeline(dataset, model_train_dim_500)

model_train_dim_1000 = train_or_load_wv(dataset[0], size=1000, window=15)
_, _ = run_pipeline(dataset, model_train_dim_1000)



Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.88
F1 on test set: 0.87
time: 256.13
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
time: 767.37
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.89
F1 on test set: 0.88
time: 1676.30


We observe that the dimension = 100 and 300 (baseline) give F1 of 0.87 while we got 0.88 from dimension = 500 and 1000. Thus it's always good to try different dimension apart of default 300 because we never know what capacity we need to carry information the model can encode. While too large capacity may not have explicit harmful side effect, it can slower the training of classification model because it requires more computational. In practice, determine the capacity which is just enough for your dataset.

# Transfer Learning

By training our word embeddings from train data, we should be aware that our word embeddings may be less quality , i.e. capture less semantic, than those pre-trained word embeddings used in Notebook 2. While those vectors are trained on several billion tokens, our word embeddings will be trained on much less dataset. We can say that our embeddings will capture the meaning of tokens more specific to domain i.e. movie reviews. But we can also say that embeddings corresponding to more common tokens will capture less semantic than those from pre-trained Word2Vec.

We can address this issue by introducing transfer learning. In high level, it's general term used when you train a model with one dataset (generally larger), then you (partially or entirely) use parameters of this trained model to train another model on another dataset (generally smaller). That new dataset and new model can be slightly different from the original model i.e. the different prediction targets or so. The use cases can be like when we train image classification from one domains and then "transfer" knowledge to another domain. We can apply similar technique by "transfer" pre-trained Word2Vec knowledge to our model. For formal definition and examples, see [this](https://towardsdatascience.com/a-comprehensive-hands-on-guide-to-transfer-learning-with-real-world-applications-in-deep-learning-212bf3b2f27a).

Let's dick a little deeper. How do we actually transfer the parameters from Word2Vec to our model?

The obvious approach is to start training the Word2Vec model with parameters from pre-trained Word2Vec, instead of small randomized parameters. Then, the second question is what are vocabularies? We can combine vocabularies from our training set with pre-trained vocabularies. However, pre-trained vocabularies size are much larger (3M vs 100k), as we discussed in Notebook 0. If we augmented 100k vocabularies with 3M and train the Word2Vec model, the effect of 100k will be very slightly. In fact, this is not necessary matter if we just want to learn embeddings for those 100 vocabularies from training set, let alone the number embeddings transfered from Word2Vec. In the experiment, we will see the effect of the size of augmented vocabularies.

We can also choose whether to freeze the embeddings transfered from pre-trained Word2Vec. If we freeze, only embeddings that are not transfered, i.e. embeddings corresponding to vocab that are in the train data but not pre-trained Word2Vec, will be trained. We will experiment both options.

Lastly, what is the number of epochs? Generally, we will train only a few epochs in transfer learning since most of the parameters are trained already and we only need introduce them to the new dataset. We will also experiment with different epochs.



## Number of augmented vocabularies

We vary the number of augmented vocabularies from 0 (only use vocabularies present in train data and transfer only embeddings of these vocabularies from Word2Vec) to 500000 (augmented vocabularies about 5 times larger than vocabularies present in train data)

In [7]:

embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    window=15,
    n_transfer=0
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()


embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    window=15,
    n_transfer=50000
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()


embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    window=15,
    n_transfer=100000
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()


embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    window=15,
    n_transfer=500000
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()

Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.88
F1 on test set: 0.88
time: 352.54
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.010769017030226619
%unk in vocab 0.40341401063785925
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.88
F1 on test set: 0.88
time: 367.48
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.008611384231879814
%unk in vocab 0.22720594237205127
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.88
F1 on test set: 0.88
time: 394.93
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.007582043581241553
%unk in vocab 0.1543357765009238
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.87
F1 on test set: 0.87
time: 449.50
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.0048322335573936295
%unk in vocab 0.036029426544190064


It turns out that the transfered knowledge from Word2Vec is not very useful for this text classification problem. We can see that the higher degree we transfered knowledge from Word2Vec (increasing the number of augmented vocabularies), the lower performance we get. Let's see results of last two experiment and we will discuss on this.

## Freeze / Not Freeze

- Freeze: keep embeddings that are transfered from pre-trained Word2Vec unchanged during training
- Unfreeze: allow embeddings that are transfered from pre-trained Word2Vec to be changed during training

In [8]:
embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    window=15,
    n_transfer=100000
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()


embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=0, 
    window=15,
    n_transfer=100000
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()

Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.88
F1 on test set: 0.88
time: 390.38
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.007582043581241553
%unk in vocab 0.1543357765009238
Best parameters set found on development set:  {'C': 100}
Best F1 on development set: 0.86
F1 on test set: 0.86
time: 304.77
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.007582043581241553
%unk in vocab 0.1543357765009238


This experiment shows that we will get better performance if we allow the transfered embeddings to be changed during training. It implies the same thing as above experiment, the less degree we rely on pre-trained Word2Vec, the better result.

## Epochs

We vary epoch from 1 to 3.

In [9]:
embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    window=15,
    n_transfer=500000,
    iter = 1
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()


embeddings = train_or_load_wv_transfer(
    dataset[0], 
    PRETRAINED_WV_MODEL_PATH, 
    lockf=1, 
    window=15,
    n_transfer=500000,
    iter = 3
)
_, vectorizer = run_pipeline(dataset, embeddings)
vectorizer.print_stat()

Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.86
F1 on test set: 0.86
time: 567.61
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.0048322335573936295
%unk in vocab 0.036029426544190064
Best parameters set found on development set:  {'C': 1000}
Best F1 on development set: 0.87
F1 on test set: 0.87
time: 514.09
fit_transform
oov freq 0.0
%unk in vocab 0.0
transform
oov freq 0.0048322335573936295
%unk in vocab 0.036029426544190064


Again, this experiment implies that the less degree we rely on pre-trained Word2Vec, the better result.

## Conclusion